In [1]:
from bs4 import BeautifulSoup as bs 
import lxml
import pandas as pd 
import numpy as np
import requests 
import time

url = "https://www.aladin.co.kr/shop/wbrowse.aspx"

payload = dict(BrowseTarget="List",ViewRowsCount="50",ViewType="detail",PublishMonth="0",SortOrder="2",page="1",Stockstatus="1",PublishDay="84",CID="55890",SearchOption="",CustReviewRankStart="",CustReviewRankEnd="",CustReviewCountStart="",CustReviewCountEnd="",PriceFilterMin="",PriceFilterMax="")

r = requests.get(url, params=payload)
response = r.text
soup = bs(response, 'lxml')
r.url

'https://www.aladin.co.kr/shop/wbrowse.aspx?BrowseTarget=List&ViewRowsCount=50&ViewType=detail&PublishMonth=0&SortOrder=2&page=1&Stockstatus=1&PublishDay=84&CID=55890&SearchOption=&CustReviewRankStart=&CustReviewRankEnd=&CustReviewCountStart=&CustReviewCountEnd=&PriceFilterMin=&PriceFilterMax='

In [2]:
data_list = []

for i in soup.select(".ss_book_list:nth-child(2n+1)"):
    if len(i.select("li")) >= 5:
        data_dict = {}
        data_dict["제목"] = i.select_one("li:nth-child(2) .bo3").text
        data_dict["책 설명"] = i.select_one("li:nth-child(2) .ss_f_g2").text if i.select_one("li:nth-child(2) .ss_f_g2") is not None else "None" 
        data_dict["글쓴이"] = i.select_one("li:nth-child(3)").text.split("|")[0]
        data_dict["출판사"] = i.select_one("li:nth-child(3)").text.split("|")[1].strip()
        data_dict["출간일"] = i.select_one("li:nth-child(3)").text.split("|")[2].strip()
        data_dict["정가"] = i.select_one("li:nth-child(4)").text.split("→")[0]
        data_dict["할인가"] = i.select_one("li:nth-child(4)").text.split("→")[1][1:9]
        data_dict["별점"] = i.select_one("li:nth-child(5) img")["src"].split("_")[1].replace("s","").replace(".gif","") if i.select_one("li:nth-child(5) img") is not None else "None"
        data_dict["리뷰수"] = i.select_one("li:nth-child(5) a").text if i.select_one("li:nth-child(5) a") is not None else "None" 
        data_dict["세일즈포인트"] = i.select_one("li:nth-child(5) b").text.strip()
        data_dict["링크"] =  i.select_one("li:nth-child(2) .bo3")["href"]
        data_list.append(data_dict)

In [23]:
data_list

[{'제목': '드루이드가 되고 싶은 당신을 위한 안내서',
  '책 설명': 'None',
  '글쓴이': '프로개 (지은이) ',
  '출판사': '드루이드아일랜드',
  '출간일': '2023년 9월',
  '정가': '24,000원 ',
  '할인가': '21,600원 ',
  '별점': '10',
  '리뷰수': '4',
  '세일즈포인트': '18,340',
  '링크': 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=324902128'},
 {'제목': '향수가 된 식물들',
  '책 설명': ' - 에르메스 조향사가 안내하는 향수 식물학의 세계',
  '글쓴이': '장 클로드 엘레나 (지은이), 카린 도어링 프로저 (그림), 이주영 (옮긴이) ',
  '출판사': '아멜리에북스',
  '출간일': '2023년 8월',
  '정가': '25,000원 ',
  '할인가': '22,500원 ',
  '별점': '10',
  '리뷰수': '91',
  '세일즈포인트': '33,440',
  '링크': 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=320434665'},
 {'제목': '완전 배출',
  '책 설명': ' - 채소·과일·무첨가 주스는 어떻게 비만과 질병을 몰아내는가',
  '글쓴이': '조승우 (지은이) ',
  '출판사': '사이몬북스',
  '출간일': '2023년 9월',
  '정가': '19,000원 ',
  '할인가': '17,100원 ',
  '별점': '8',
  '리뷰수': '7',
  '세일즈포인트': '22,265',
  '링크': 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=322658545'},
 {'제목': '건강과 다이어트를 동시에 잡는 7대 3의 법칙 채소·과일식',
  '책 설명': ' - 단순하면서 자연스러운 가장 효과적인 식단',
  '글쓴이': '조승우 (지은이) '

In [24]:
df = pd.DataFrame(data_list)

In [25]:
url = df["링크"]

In [29]:
detail_list = []

for urls in url[:15]:
    detail_get = requests.get(urls)
    detail_soup = bs(detail_get.text,"lxml")
    contents = detail_soup.select(".Ere_prod_middlewrap")
    book_id = urls.split("=")[-1]
    print(f"현재 {urls} 크롤링 중")
    for i in contents:
        detail_dict = {}
        bookId = book_id
        pages = [i.text for i in i.select(".conts_info_list1 li")][0]
        size = [i.text for i in i.select(".conts_info_list1 li")][1]
        weight = [i.text for i in i.select(".conts_info_list1 li")][2]
        category = [i.text.replace("\xa0", "").split(">") for i in i.select("#ulCategory li")]
        detail_dict["ID"] = bookId
        detail_dict["페이지"] = pages
        detail_dict["사이즈"] = size
        detail_dict["카테고리"] = category
        detail_list.append(detail_dict)
    time.sleep(5)

In [30]:
detail_list

[{'ID': '324902128',
  '페이지': '400쪽',
  '사이즈': '152*223mm (A5신)',
  '카테고리': [['국내도서', '건강/취미', '원예', '화훼 접기'],
   ['국내도서', '과학', '동물과 식물', '식물 일반 접기'],
   ['국내도서', '요리/살림', '원예/조경/텃밭 접기']]},
 {'ID': '320434665',
  '페이지': '양장본',
  '사이즈': '228쪽',
  '카테고리': [['국내도서', '에세이', '외국에세이 접기'],
   ['국내도서', '건강/취미', '스포츠/레저 기타', '기타 접기'],
   ['국내도서', '과학', '동물과 식물', '꽃과 나무 접기'],
   ['국내도서', '과학', '동물과 식물', '식물 일반 접기'],
   ['국내도서', '예술/대중문화', '미술', '화집 접기'],
   ['국내도서', '요리/살림', '원예/조경/텃밭 접기']]},
 {'ID': '322658545',
  '페이지': '304쪽',
  '사이즈': '145*210mm',
  '카테고리': [['국내도서', '건강/취미', '건강정보', '건강에세이/건강정보 접기'],
   ['국내도서', '요리/살림', '건강요리 접기']]},
 {'ID': '302583748',
  '페이지': '294쪽',
  '사이즈': '150*220mm',
  '카테고리': [['국내도서', '건강/취미', '건강정보', '음식과 건강 접기'],
   ['국내도서', '건강/취미', '건강정보', '건강에세이/건강정보 접기'],
   ['국내도서', '요리/살림', '건강요리 접기'],
   ['국내도서', '요리/살림', '다이어트 요리 접기']]},
 {'ID': '324921932',
  '페이지': '208쪽',
  '사이즈': '188*255mm',
  '카테고리': [['국내도서', '건강/취미', '글씨 쓰기 접기']]},
 {'ID': '319559579',
  '페이지'

In [41]:
#함수형으로 변환

def scrape_main_aladin(url, payload):
    r = requests.get(url, params=payload)
    response = r.text
    soup = bs(response, 'lxml')
    
    data_list = []
    for i in soup.select(".ss_book_list:nth-child(2n+1)"):
        if len(i.select("li")) >= 5:
            data_dict = {}
            data_dict["제목"] = i.select_one("li:nth-child(2) .bo3").text
            data_dict["책 설명"] = i.select_one("li:nth-child(2) .ss_f_g2").text if i.select_one("li:nth-child(2) .ss_f_g2") is not None else "None" 
            data_dict["글쓴이"] = i.select_one("li:nth-child(3)").text.split("|")[0]
            data_dict["출판사"] = i.select_one("li:nth-child(3)").text.split("|")[1].strip()
            data_dict["출간일"] = i.select_one("li:nth-child(3)").text.split("|")[2].strip()
            data_dict["정가"] = i.select_one("li:nth-child(4)").text.split("→")[0]
            data_dict["할인가"] = i.select_one("li:nth-child(4)").text.split("→")[1][1:9]
            data_dict["별점"] = i.select_one("li:nth-child(5) img")["src"].split("_")[1].replace("s","").replace(".gif","") if i.select_one("li:nth-child(5) img") is not None else "None"
            data_dict["리뷰수"] = i.select_one("li:nth-child(5) a").text if i.select_one("li:nth-child(5) a") is not None else "None" 
            data_dict["세일즈포인트"] = i.select_one("li:nth-child(5) b").text.strip()
            data_dict["링크"] =  i.select_one("li:nth-child(2) .bo3")["href"]
            data_list.append(data_dict)

    return data_list

def main_page_scrapy(url):
    detail_list = []

    for urls in url[:15]:
        detail_get = requests.get(urls)
        detail_soup = bs(detail_get.text,"lxml")
        contents = detail_soup.select(".Ere_prod_middlewrap")
        book_id = urls.split("=")[-1]
        print(f"현재 {urls} 크롤링 중")
        for i in contents:
            detail_dict = {}
            bookId = book_id
            pages = [i.text for i in i.select(".conts_info_list1 li")][0]
            size = [i.text for i in i.select(".conts_info_list1 li")][1]
            weight = [i.text for i in i.select(".conts_info_list1 li")][2]
            category = [i.text.replace("\xa0", "").split(">") for i in i.select("#ulCategory li")]
            detail_dict["ID"] = bookId
            detail_dict["페이지"] = pages
            detail_dict["사이즈"] = size
            detail_dict["카테고리"] = category
            detail_list.append(detail_dict)
        time.sleep(5)
    
    return detail_list 

def main():
    url = "https://www.aladin.co.kr/shop/wbrowse.aspx"
    payload = dict(BrowseTarget="List", ViewRowsCount="50", ViewType="detail", PublishMonth="0", SortOrder="2", page="1", Stockstatus="1", PublishDay="84", CID="55890", SearchOption="", CustReviewRankStart="", CustReviewRankEnd="", CustReviewCountStart="", CustReviewCountEnd="", PriceFilterMin="", PriceFilterMax="")
    book_data = scrape_main_aladin(url, payload)
    df = pd.DataFrame(book_data)
    book_urls = df["링크"] 
    book_details = main_page_scrapy(book_urls)
    
    
if __name__ == "__main__":
    main()


현재 https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=324902128 크롤링 중
현재 https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=320434665 크롤링 중
현재 https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=322658545 크롤링 중
현재 https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=302583748 크롤링 중
현재 https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=324921932 크롤링 중
현재 https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=319559579 크롤링 중
현재 https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=324225465 크롤링 중
현재 https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=323783289 크롤링 중
현재 https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=307204828 크롤링 중
현재 https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=324710543 크롤링 중
현재 https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=321617787 크롤링 중
현재 https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=323558091 크롤링 중
현재 https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=297128441 크롤링 중
현재 https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=322802225 크롤링 중


KeyboardInterrupt: 